<a href="https://colab.research.google.com/github/aditya-saurabh/generative-adversial-network/blob/master/CycleGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# importing libraries
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt

In [11]:
# defining discriminator model

def discriminator(image_shape):
  # weight initialization for layers
  init = tf.keras.initializers.RandomNormal(stddev=0.02)

  # input layer
  input_layer = tf.keras.layers.Input(shape=image_shape)

  # C64
  d = tf.keras.layers.Conv2D(64, (4,4), (2,2), padding='same', kernel_initializer=init)(input_layer)
  d = tf.keras.layers.LeakyReLU(alpha=0.2)(d)

  # C128
  d = tf.keras.layers.Conv2D(128, (4,4), (2,2), padding='same', kernel_initializer=init)(d)
  d = tfa.layers.InstanceNormalization(axis=-1)(d)
  d = tf.keras.layers.LeakyReLU(alpha=0.2)(d)

  # C256
  d = tf.keras.layers.Conv2D(256, (4,4), (2,2), padding='same', kernel_initializer=init)(d)
  d = tfa.layers.InstanceNormalization(axis=-1)(d)
  d = tf.keras.layers.LeakyReLU(alpha=0.2)(d)

  # C512
  d = tf.keras.layers.Conv2D(512, (4,4), (2,2), padding='same', kernel_initializer=init)(d)
  d = tfa.layers.InstanceNormalization(axis=-1)(d)
  d = tf.keras.layers.LeakyReLU(alpha=0.2)(d)

  # C512 - second last layer
  d = tf.keras.layers.Conv2D(512, (4,4), (1,1), padding='same', kernel_initializer=init)(d)
  d = tfa.layers.InstanceNormalization(axis=-1)(d)
  d = tf.keras.layers.LeakyReLU(alpha=0.2)(d)

  # patch layer -- last layer
  patch_out = tf.keras.layers.Conv2D(1, (4,4), (1,1), padding='same', kernel_initializer=init)(d)

  model = tf.keras.Model(input_layer, patch_out)

  # compiling the model
  opt = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
  model.compile(optimizer=opt, loss='mse', loss_weights=[0.5])

  return model

In [12]:
# printing model summary of discriminator

image_shape = (256,256,3)
dis = discriminator(image_shape)
dis.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 128, 128, 64)      3136      
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 64, 64, 128)       131200    
_________________________________________________________________
instance_normalization_12 (I (None, 64, 64, 128)       256       
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 32, 32, 256)       5245

In [25]:
# defining a resnet block

def res_block(n_filter, input_layer):
  #weight initialization
  init = tf.keras.initializers.RandomNormal(stddev=0.02)

  #first layer convolutional layer
  g = tf.keras.layers.Conv2D(n_filter, (3,3), (1,1), padding='same', kernel_initializer=init)(input_layer)
  g = tfa.layers.InstanceNormalization(axis=-1)(g)
  g = tf.keras.layers.ReLU()(g)

  #second layer convolutional layer
  g = tf.keras.layers.Conv2D(n_filter, (3,3), (1,1), padding='same', kernel_initializer=init)(g)
  g = tfa.layers.InstanceNormalization(axis=-1)(g)
  
  #concatenating
  merged = tf.keras.layers.Concatenate()([g, input_layer])

  return merged

In [28]:
# define generator model

def generator(input_shape=(256,256,3), n_resnet=9):
  # weight initialization
  init = tf.keras.initializers.RandomNormal(stddev=0.02)

  # input layer
  input_layer = tf.keras.layers.Input(shape=input_shape)

  #C64-F7-S1-P'same' I ReLU
  g = tf.keras.layers.Conv2D(64, (7,7), padding='same', kernel_initializer=init)(input_layer)
  g = tfa.layers.InstanceNormalization(axis=-1)(g)
  g = tf.keras.layers.Activation('relu')(g)

  #C128-F3-S2-P'same' I ReLU
  g = tf.keras.layers.Conv2D(128, (3,3), (2,2),padding='same', kernel_initializer=init)(g)
  g = tfa.layers.InstanceNormalization(axis=-1)(g)
  g = tf.keras.layers.Activation('relu')(g)

  #C256-F3-S2-P'same' I ReLU
  g = tf.keras.layers.Conv2D(256, (3,3), (2,2), padding='same', kernel_initializer=init)(g)
  g = tfa.layers.InstanceNormalization(axis=-1)(g)
  g = tf.keras.layers.Activation('relu')(g)

  #Residual blocks
  for _ in range(n_resnet):
    g = res_block(256, g)
  
  #UpC128-F3-S2-P'same' I ReLU
  g = tf.keras.layers.Conv2DTranspose(128, (3,3), (2,2), padding='same', kernel_initializer=init)(g)
  g = tfa.layers.InstanceNormalization(axis=-1)(g)
  g = tf.keras.layers.Activation('relu')(g)

  #UpC64-F3-S2-P'same' I ReLU
  g = tf.keras.layers.Conv2DTranspose(64, (3,3), (2,2), padding='same', kernel_initializer=init)(g)
  g = tfa.layers.InstanceNormalization(axis=-1)(g)
  g = tf.keras.layers.Activation('relu')(g)

  #C3-F7-S0-P'same' I tanh
  g = tf.keras.layers.Conv2D(3, (7,7), padding='same', kernel_initializer=init)(g)
  g = tfa.layers.InstanceNormalization(axis=-1)(g)
  out_img = tf.keras.layers.Activation('tanh')(g)

  #model
  model = tf.keras.models.Model(input_layer, out_img)

  return model

In [29]:
# calling generator
gen = generator()

# printing the model summary
gen.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_60 (Conv2D)              (None, 256, 256, 64) 9472        input_11[0][0]                   
__________________________________________________________________________________________________
instance_normalization_53 (Inst (None, 256, 256, 64) 128         conv2d_60[0][0]                  
__________________________________________________________________________________________________
activation_15 (Activation)      (None, 256, 256, 64) 0           instance_normalization_53[0][0]  
____________________________________________________________________________________________